In [24]:
from torch import nn, optim
import numpy as np
import pickle as p
import torch
from tqdm.notebook import trange
import tqdm

from network import ConvNet
from sklearn.metrics import f1_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataset import CogDataset

In [25]:
# function for training and evaluation
def train(model, num_epochs):
    losses_train = []
    accuracies_train = []
    f1_train = []
    losses_val = []
    accuracies_val = []
    f1_val = []

    for epoch in trange(num_epochs, unit="epochs"):

        # training
        running_loss = 0.0
        running_accuracy = 0.0
        running_f1 = 0.0

        model.train()

        input, target = next(iter(train_dataloader))

        optimizer.zero_grad()

        output = model(input)
        loss = criterion(output, target)
        loss.backward()

        running_loss += loss.item()
        for o, t in zip(output, target):
            if o == t:
               running_acurracy+=1 
        running_f1 += f1_score(target, output)
    
    optimizer.step()

    running_loss /= len(train_dataloader)
    running_accuracy /= len(train_dataloader)
    running_f1 /= len(train_dataloader)

    losses_train.append(running_loss)
    accuracies_train.append(running_accuracy)
    f1_train.append(running_f1)

     # output
    tqdm.write('Epoch {} (train) -- loss: {:.4f} f1: {:.4f}'.format(epoch, running_loss, f1))


    # validation
    with torch.no_grad():

        model.eval()

        running_loss = 0.0
        running_accuracy = 0.0
        running_f1 = 0.0

        for input, target in val_dataloader:

            output = model(input)
            loss = criterion(output, target)

            running_loss += loss.item()
            for o, t in zip(output, target):
                if o == t:
                    running_acurracy+=1 
            running_f1 += f1_score(target, output)

        running_loss /= len(val_dataloader)
        running_accuracy /= len(val_dataloader)
        running_f1 /= len(val_dataloader)

        losses_val.append(running_loss)
        accuracies_val.append(running_accuracy)
        f1_val.append(running_f1)

        # output
        tqdm.write('Epoch {} (valid) -- loss: {:.4f} f1: {:.4f}'.format(epoch, running_loss, f1))
    # make model callable outside this function
    train.trained_model = model


    
    


In [21]:
# load data and label files
train_files = torch.from_numpy(p.load(open(r"data\train.pkl", "rb")))
label_files = torch.from_numpy(p.load(open(r"data\labels.pkl", "rb")))


# create dataset
train_dataset = CogDataset(train=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=3, shuffle=True)

torch.Size([2284])


In [ ]:
# define loss, optimizer etc.
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# train model
train(model, 20)